# **Setup**

In [11]:
!pip install -qq langchain
!pip install -qq langchain-community
!pip install -qq langchain-google-genai

In [12]:
import os

In [ ]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

In [18]:
os.environ['GOOGLE_API_KEY'] = GEMINI_API_KEY


In [19]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3,google_api_key=GEMINI_API_KEY)

# **Single Chain (Sequential)**

The simplest chain: Prompt → LLM → Parser

In [20]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Summarize this text in one sentence:\n\n{text}"
)

parser = StrOutputParser()

# Compose pipeline using |
chain = prompt | llm | parser

result = chain.invoke({"text": "LangChain helps developers build apps powered by LLMs with prompts, chains, memory, and agents."})
print(result)


LangChain simplifies the development of LLM-powered applications by providing tools for managing prompts, chaining operations, implementing memory, and creating autonomous agents.


# **Parallel Chains (RunnableParallel)**

Run multiple chains (or models) at the same time → returns a dict of results.

In [22]:
from langchain_core.runnables import RunnableParallel

# Two different prompt chains
summary_prompt = ChatPromptTemplate.from_template("Summarize in 10 words:\n\n{text}")
keywords_prompt = ChatPromptTemplate.from_template("Extract 3 keywords:\n\n{text}")

summary_chain = summary_prompt | llm | parser
keywords_chain = keywords_prompt | llm | parser

# Run them in parallel
parallel_chain = RunnableParallel(
    summary=summary_chain,
    keywords=keywords_chain
)

result = parallel_chain.invoke({
    "text": "LangChain simplifies working with LLMs by providing abstractions for prompts, chains, memory, and agents."
})

print(result)

{'summary': 'LangChain simplifies LLM interaction via modular prompts, chains, memory, and agents.', 'keywords': 'LangChain, LLMs, Agents'}


# **Conditional Chains (RunnableBranch)**

Choose which chain to run depending on input conditions.

In [ ]:
from langchain_core.runnables import RunnableBranch

# Prompt for short vs long text
short_prompt = ChatPromptTemplate.from_template("Summarize briefly: {text}")
long_prompt = ChatPromptTemplate.from_template("Give a detailed explanation of: {text}")

short_chain = short_prompt | llm | parser
long_chain = long_prompt | llm | parser

# Branch logic
conditional_chain = RunnableBranch(
    (lambda x: len(x["text"].split()) < 10, short_chain),
    (long_chain)  # default case
)

print(conditional_chain.invoke({"text": "AI changes everything."}))
print(conditional_chain.invoke({"text": "LangChain provides abstractions like prompts, chains, agents, and memory to make building LLM apps much easier."}))
